In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
def get_counts(data, regions, s, mode = None): # modes: None/all, picks, presence
    # initialize a dictionary for each region, each of which will contain the counts for each champion
    game_counts = {}
    champ_counts = {}
    for region in regions:
        for year in years:
            for season in seasons:
                champ_counts[(region, year, season)] = {}
                game_counts[(region, year, season)] = 0
    
    if mode is None or mode.lower() == 'all':
        key1 = lambda champ: (champ, 'b')
        key2 = lambda champ: (champ, 'p')
    else:
        key1 = lambda champ: champ
        key2 = lambda champ: champ

    # loop through rows, incrementing the respective counters for all champions in that game
    for idx, row in data.iterrows():
        region, year, season = row[0:3]
        game_counts[(region, year, season)] += 1
        
        if not mode == 'picks':
            for champ in [x.strip() for x in row[7].replace("'", "").replace('[', '').replace(']', '').split(',')]: # for every champion banned by blue
                if key1(champ) in champ_counts[(region, year, season)].keys():
                    count, wins = champ_counts[(region, year, season)][key1(champ)]
                    champ_counts[(region, year, season)][key1(champ)] = (count + 1, wins + row[4])
                else: # intialize new entry in dictionary, if necessary
                    champ_counts[(region, year, season)][key1(champ)] = (1, row[4])

        for champ in row[9:14]: # for every champion present on blue
            if key2(champ) in champ_counts[(region, year, season)].keys():
                count, wins = champ_counts[(region, year, season)][key2(champ)]
                champ_counts[(region, year, season)][key2(champ)] = (count + 1, wins + row[4])
            else: # intialize new entry in dictionary, if necessary
                champ_counts[(region, year, season)][key2(champ)] = (1, row[4])
        
        if not mode == 'picks':
            for champ in [x.strip() for x in row[8].replace("'", "").replace('[', '').replace(']', '').split(',')]: # for every champion banned by red
                if key1(champ) in champ_counts[(region, year, season)].keys():
                    count, wins = champ_counts[(region, year, season)][key1(champ)]
                    champ_counts[(region, year, season)][key1(champ)] = (count + 1, wins + row[5])
                else: # intialize new entry in dictionary, if necessary
                    champ_counts[(region, year, season)][key1(champ)] = (1, row[5])

        for champ in row[14:]: # for every champion present on red
            if key2(champ) in champ_counts[(region, year, season)].keys():
                count, wins = champ_counts[(region, year, season)][key2(champ)]
                champ_counts[(region, year, season)][key2(champ)] = (count + 1, wins + row[5])
            else: # intialize new entry in dictionary, if necessary
                champ_counts[(region, year, season)][key2(champ)] = (1, row[5])
        
    freq_champ_counts = {}
    for region, counts in champ_counts.items():
        freq_champ_counts[region] = {}
        for champ, vals in counts.items():
            if vals[0]/game_counts[region] >= s:
                freq_champ_counts[region][champ] = vals

    # initialize a dictionary for each region, each of which will contain the counts for each champion pair
    champ_pair_counts = {}
    for region in regions:
        for year in years:
            for season in seasons:
                champ_pair_counts[(region, year, season)] = {}
        
    # loop through rows, incrementing the respective counters for all champions in that game
    for idx, row in data.iterrows():
        region, year, season = row[0:3]
        b_basket = []
        r_basket = []

        for champ in [x.strip() for x in row[7].replace("'", "").replace('[', '').replace(']', '').split(',')]: # for every champion banned by blue
            b_basket.append(key1(champ))
        for champ in row[9:14]: # for every champion present on blue
            b_basket.append(key2(champ))
        for champ in [x.strip() for x in row[8].replace("'", "").replace('[', '').replace(']', '').split(',')]: # for every champion banned by red
            r_basket.append(key1(champ))
        for champ in row[14:]: # for every champion present on red
            r_basket.append(key2(champ))

        for i in range(len(b_basket)):
            for j in range(i + 1, len(b_basket)):
                key = tuple(sorted([b_basket[i], b_basket[j]]) + ['same'])
                if key in champ_pair_counts[(region, year, season)].keys():
                    count, wins = champ_pair_counts[(region, year, season)][key]
                    champ_pair_counts[(region, year, season)][key] = (count + 1, wins + row[4])
                else: # intialize new entry in dictionary, if necessary
                    champ_pair_counts[(region, year, season)][key] = (1, row[4])

        for i in range(len(r_basket)):
            for j in range(i + 1, len(r_basket)):
                key = tuple(sorted([r_basket[i], r_basket[j]]) + ['same'])
                if key in champ_pair_counts[(region, year, season)].keys():
                    count, wins = champ_pair_counts[(region, year, season)][key]
                    champ_pair_counts[(region, year, season)][key] = (count + 1, wins + row[4])
                else: # intialize new entry in dictionary, if necessary
                    champ_pair_counts[(region, year, season)][key] = (1, row[4])

        for i in range(len(b_basket)):
            for j in range(len(r_basket)):
                if b_basket[i] > r_basket[j]:
                    flip = True
                    key = (r_basket[j], b_basket[i], 'opp')
                else:
                    flip = False
                    key = (b_basket[i], r_basket[j], 'opp')
                if key in champ_pair_counts[(region, year, season)].keys():
                    count, wins1, wins2 = champ_pair_counts[(region, year, season)][key]
                    if flip:
                        champ_pair_counts[(region, year, season)][key] = (count + 1, wins1 + row[5], wins2 + row[4])
                    else:
                        champ_pair_counts[(region, year, season)][key] = (count + 1, wins1 + row[4], wins2 + row[5])
                else: # intialize new entry in dictionary, if necessary
                    champ_pair_counts[(region, year, season)][key] = (1, row[4], row[5])
        
    freq_champ_pair_counts = {}
    for region, counts in champ_pair_counts.items():
        freq_champ_pair_counts[region] = {}
        for champ, vals in counts.items():
            if vals[0]/game_counts[region] >= s:
                freq_champ_pair_counts[region][champ] = vals
    
    return game_counts, freq_champ_counts, freq_champ_pair_counts

In [3]:
def print_sorted(count_dict, game_counts = None):
    for region, counts in count_dict.items():
        champ_list = np.array(list(counts.keys()))
        if len(champ_list) < 1:
            continue
        
        # get each values of counts and wins
        vals = np.array(list(counts.values()))
        count_vals = np.array([val[0] for val in vals]) 
        win_vals = np.array([val[1:] for val in vals])
        
        # indices to sort in descending order, according to count
        idx = np.flip(np.argsort(count_vals))
        champ_list = champ_list[idx]
        count_vals = count_vals[idx]
        win_vals = win_vals[idx]
        
        # print header
        if game_counts is None:
            print(region)
        else:
            print('{} ({} games)'.format(' '.join([str(r) for r in region]), game_counts[region]))
        
        # print Champion and information in descending order, according to count
        for i in range(len(champ_list)):
            print(champ_list[i], count_vals[i], win_vals[i])
        print('-----------------------------------------------')

In [4]:
def get_vecs(count_dict_list, game_counts):
    champ_list = set()
    regions = []
    
    # allow for multiple count dictionaries to be provided at once (can have a column for each champion AND champion pair, and so on)
    if not isinstance(count_dict_list, list):
        count_dict_list = [count_dict_list]
    
    # get sorted lists of all champions and leagues/seasons in the provided data
    for count_dict in count_dict_list:
        for region, counts in count_dict.items():
            champs = set(list(counts.keys()))
            if len(champs) < 1:
                continue
            if region not in regions:
                regions.append(region)
            champ_list = champ_list.union(champs)
    champ_list = sorted(list(champ_list))
    regions = sorted(regions)
    
    # each row corresponds to a league/season pair, each column corresponds to an instance of a champion
    m = len(regions)
    n = len(champ_list)
    vecs = np.zeros((m, n))
    
    # assign appropriate values to the matrix
    for i in range(m):
        for j in range(n):
            # somewhat convoluted way of getting the appropriate value from the dictionary it appears in
            vecs[i, j] =  np.sum([count_dict[regions[i]].get(champ_list[j], [0])[0]/game_counts[regions[i]] for count_dict in count_dict_list])
    
    return vecs, champ_list, regions

In [5]:
def k_means(data, k, tol = 1e-6, max_iter = 1e8, verbose = False):
    # randomly assign centroids to existing data points
    idx = np.random.choice(list(range(data.shape[0])), k)
    centroids = data[idx]
    
    dists = [None for _ in range(k)]
    clusters = np.array([-1 for _ in range(len(data))])
    
    change = math.inf
    num_iter = 0
    while change > tol and num_iter < max_iter:
        num_iter += 1
        #update classifications
        for j in range(len(data)):
            dists = [np.linalg.norm(data[j,:] - cent) for cent in centroids]
            assignment = np.argmin(dists)
            clusters[j] = assignment

        #update centroids
        centroids_new = np.zeros(centroids.shape)
        changes = []
        for j in range(k):
            # check if a cluster is empty, and return None if this is the case
            if len(data[clusters==j]) == 0:
                if verbose:
                    print('Cluster with no data points.')
                return None, None
            centroids_new[j]= np.array([np.sum(data[clusters==j,i])/len(data[clusters==j,i]) for i in range(data.shape[1])])
            changes.append(np.linalg.norm(centroids_new[j] - centroids[j]))
        change = np.max(changes)
        centroids = centroids_new
        
    if num_iter == max_iter:
        print('Maximum number of iterations reached.')

    return centroids, clusters

In [6]:
def grid_plot_clusters(regions, clusters, title = 'Something'):
    # get values for grid axes
    leagues = sorted(list(set([row[0] for row in regions])))
    seasons = sorted(list(set([str(row[1]) + ' ' + str(row[2]) for row in regions])))
    
    # get values for actual grid to be plotted
    values = np.zeros((len(leagues), len(seasons)))
    for i in range(len(clusters)):
        row = regions[i]
        values[leagues.index(row[0]), seasons.index(str(row[1]) + ' ' + str(row[2]))] = clusters[i] + 1
            # +1 to distinguish cluster = 0 case from the case where no data was present
    
    fig, ax = plt.subplots(figsize = (30, 10))
    im = ax.imshow(values, aspect = 1/2, cmap = 'tab20')
    
    # set axis ticks and labels
    ax.set_xticks(np.arange(len(seasons)))
    ax.set_yticks(np.arange(len(leagues)))
    ax.set_xticklabels(seasons)
    ax.set_yticklabels(leagues)

    # rotate tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    
    ax.set_title(title)
    plt.show()

In [7]:
data = pd.read_csv('.\data\LeagueofLegends.csv')
data.head()

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...",...,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",LemonNation,Thresh,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']",http://matchhistory.na.leagueoflegends.com/en/...
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...",...,Azir,"[475, 475, 552, 786, 1097, 1389, 1660, 1955, 2...",CoreJJ,Corki,"[475, 475, 532, 868, 1220, 1445, 1732, 1979, 2...",KiWiKiD,Annie,"[515, 515, 583, 752, 900, 1066, 1236, 1417, 15...","['RekSai', 'Janna', 'Leblanc']",http://matchhistory.na.leagueoflegends.com/en/...
2,NALCS,2015,Spring,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...",...,Azir,"[475, 475, 533, 801, 1006, 1233, 1385, 1720, 1...",Cop,Corki,"[475, 475, 533, 781, 1085, 1398, 1782, 1957, 2...",BunnyFuFuu,Janna,"[515, 515, 584, 721, 858, 1002, 1168, 1303, 14...","['Leblanc', 'Zed', 'RekSai']",http://matchhistory.na.leagueoflegends.com/en/...
3,NALCS,2015,Spring,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...",...,Lulu,"[475, 475, 532, 771, 1046, 1288, 1534, 1776, 2...",KEITH,KogMaw,"[475, 475, 532, 766, 1161, 1438, 1776, 1936, 2...",Xpecial,Janna,"[515, 515, 583, 721, 870, 1059, 1205, 1342, 15...","['RekSai', 'Rumble', 'LeeSin']",http://matchhistory.na.leagueoflegends.com/en/...
4,NALCS,2015,Spring,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20...",...,Lulu,"[475, 475, 532, 807, 1042, 1338, 1646, 1951, 2...",Maplestreet8,Corki,"[475, 475, 532, 792, 1187, 1488, 1832, 2136, 2...",Dodo8,Annie,"[475, 475, 538, 671, 817, 948, 1104, 1240, 136...","['Rumble', 'Sivir', 'Rengar']",http://matchhistory.na.leagueoflegends.com/en/...


In [8]:
positions = ['Top', 'Jungle', 'Middle', 'ADC', 'Support']

all_regions = data['League'].unique()
major_regions = ['EULCS', 'LCK', 'NALCS', 'WC'] # EULCS - Europe, LCK - Korea, NALCS - US/Canada, WC - World Championship; unfortunately, China not included in the dataset
years = data['Year'].unique()
seasons = data['Season'].unique()
teams = list(set(data['blueTeamTag'].unique()).union(set(data['redTeamTag'].unique())))

In [9]:
cols = [team + pos + 'Champ' for team in ['blue', 'red'] for pos in positions]

data_less = data[['League', 'Year', 'Season', 'blueTeamTag', 'bResult', 'rResult', 'redTeamTag', 'blueBans', 'redBans'] + cols]

data_condensed = data[(data['Type'].isin(['International', 'Season'])) & (data['Year'] != 2018)][['League', 'Year', 'Season', 'blueTeamTag', 'bResult', 'rResult', 'redTeamTag', 'blueBans', 'redBans'] + cols]
data_condensed = data_condensed[data_condensed['League'].isin(major_regions)]
data_condensed.head()

,League,Year,Season,blueTeamTag,bResult,rResult,redTeamTag,blueBans,redBans,blueTopChamp,blueJungleChamp,blueMiddleChamp,blueADCChamp,blueSupportChamp,redTopChamp,redJungleChamp,redMiddleChamp,redADCChamp,redSupportChamp
0,NALCS,2015,Spring,TSM,1,0,C9,"['Rumble', 'Kassadin', 'Lissandra']","['Tristana', 'Leblanc', 'Nidalee']",Irelia,RekSai,Ahri,Jinx,Janna,Gnar,Elise,Fizz,Sivir,Thresh
1,NALCS,2015,Spring,CST,0,1,DIG,"['Kassadin', 'Sivir', 'Lissandra']","['RekSai', 'Janna', 'Leblanc']",Gnar,Rengar,Ahri,Caitlyn,Leona,Irelia,JarvanIV,Azir,Corki,Annie
2,NALCS,2015,Spring,WFX,1,0,GV,"['JarvanIV', 'Lissandra', 'Kassadin']","['Leblanc', 'Zed', 'RekSai']",Renekton,Rengar,Fizz,Sivir,Annie,Sion,LeeSin,Azir,Corki,Janna
3,NALCS,2015,Spring,TIP,0,1,TL,"['Annie', 'Lissandra', 'Kassadin']","['RekSai', 'Rumble', 'LeeSin']",Irelia,JarvanIV,Leblanc,Sivir,Thresh,Gnar,Nunu,Lulu,KogMaw,Janna
4,NALCS,2015,Spring,CLG,1,0,T8,"['Irelia', 'Pantheon', 'Kassadin']","['Rumble', 'Sivir', 'Rengar']",Gnar,JarvanIV,Lissandra,Tristana,Janna,Sion,RekSai,Lulu,Corki,Annie


# A-Priori
Below we apply a-priori to find common champions in each of the major regions during each season, as well as frequent combinations of champion picks, bans, or both.

In [10]:
game_counts, champ_counts, champ_pair_counts = get_counts(data_condensed, major_regions, 0.20, 'presence') # hundreds of lines of (possibly inefficient) code, reduced to a single line

In [11]:
print_sorted(champ_counts, game_counts)

EULCS 2015 Spring (91 games)
RekSai 72 [30]
Zed 66 [35]
Ahri 62 [34]
Leblanc 58 [27]
Graves 57 [27]
JarvanIV 56 [26]
Gnar 56 [29]
Kassadin 56 [33]
Annie 55 [22]
Lissandra 51 [25]
Nidalee 49 [29]
Maokai 44 [25]
Cassiopeia 44 [29]
Thresh 44 [19]
Lulu 42 [21]
Sivir 40 [22]
Corki 38 [14]
Morgana 37 [21]
LeeSin 34 [16]
Irelia 33 [14]
Leona 31 [14]
Janna 29 [13]
Kalista 28 [17]
Rengar 25 [15]
Azir 25 [9]
Sejuani 22 [12]
Rumble 20 [8]
Jax 19 [14]
-----------------------------------------------
EULCS 2015 Summer (91 games)
Gragas 81 [46]
RekSai 80 [34]
Kalista 78 [38]
Alistar 74 [35]
Sivir 71 [36]
Ryze 69 [30]
Rumble 53 [21]
Morgana 51 [31]
Azir 50 [25]
Thresh 49 [27]
Shen 39 [18]
Corki 39 [20]
Maokai 36 [15]
Jayce 35 [15]
Fizz 33 [17]
Lulu 32 [19]
Ekko 31 [13]
Viktor 29 [14]
Varus 28 [17]
Leblanc 26 [16]
Gnar 25 [14]
KogMaw 25 [16]
Nidalee 23 [15]
Hecarim 23 [5]
Nautilus 23 [10]
Vladimir 22 [11]
Jax 20 [12]
Janna 19 [11]
-----------------------------------------------
EULCS 2016 Spring (90 ga

In [12]:
print_sorted(champ_pair_counts, game_counts)

EULCS 2015 Spring (91 games)
['RekSai' 'Zed' 'same'] 32 (17,)
['Ahri' 'RekSai' 'opp'] 31 (19, 12)
['Gnar' 'RekSai' 'same'] 30 (21,)
['JarvanIV' 'RekSai' 'opp'] 30 (16, 14)
['Gnar' 'Lissandra' 'opp'] 28 (14, 14)
['Kassadin' 'Lissandra' 'opp'] 26 (15, 11)
['Ahri' 'Zed' 'opp'] 26 (13, 13)
['Lissandra' 'RekSai' 'opp'] 26 (15, 11)
['LeeSin' 'RekSai' 'opp'] 25 (14, 11)
['Graves' 'RekSai' 'opp'] 25 (15, 10)
['Gnar' 'Kassadin' 'opp'] 25 (12, 13)
['RekSai' 'Zed' 'opp'] 24 (8, 16)
['Lissandra' 'Zed' 'opp'] 24 (10, 14)
['Kassadin' 'Zed' 'opp'] 24 (10, 14)
['Graves' 'Zed' 'opp'] 24 (11, 13)
['JarvanIV' 'Lissandra' 'same'] 24 (15,)
['Ahri' 'Graves' 'same'] 24 (15,)
['Kassadin' 'RekSai' 'opp'] 23 (16, 7)
['Gnar' 'Zed' 'same'] 23 (16,)
['Gnar' 'JarvanIV' 'opp'] 23 (12, 11)
['JarvanIV' 'Kassadin' 'opp'] 23 (11, 12)
['Leblanc' 'RekSai' 'opp'] 23 (12, 11)
['Nidalee' 'Zed' 'opp'] 23 (12, 11)
['Annie' 'RekSai' 'opp'] 23 (13, 10)
['Graves' 'JarvanIV' 'opp'] 22 (12, 10)
['Leblanc' 'RekSai' 'same'] 22 (12,)


['Gragas' 'Vladimir' 'opp'] 38 (24, 14)
['Ryze' 'Trundle' 'opp'] 37 (15, 22)
['Nidalee' 'RekSai' 'opp'] 37 (22, 15)
['Karma' 'RekSai' 'opp'] 37 (17, 20)
-----------------------------------------------
EULCS 2017 Spring (156 games)
['Camille' 'Malzahar' 'opp'] 85 (36, 49)
['Camille' 'Rengar' 'same'] 77 (44,)
['Rengar' 'Shen' 'opp'] 76 (37, 39)
['Leblanc' 'Rengar' 'same'] 76 (45,)
['Malzahar' 'Rengar' 'opp'] 73 (43, 30)
['Rengar' 'Zyra' 'opp'] 72 (31, 41)
['Camille' 'Shen' 'opp'] 72 (33, 39)
['Camille' 'Leblanc' 'same'] 71 (45,)
['Leblanc' 'Zyra' 'opp'] 68 (31, 37)
['Camille' 'Rengar' 'opp'] 66 (34, 32)
['Leblanc' 'Shen' 'opp'] 64 (32, 32)
['Camille' 'Varus' 'same'] 63 (42,)
['Leblanc' 'Malzahar' 'opp'] 63 (23, 40)
['Leblanc' 'Varus' 'same'] 61 (38,)
['Malzahar' 'Shen' 'opp'] 61 (33, 28)
['Camille' 'Jhin' 'opp'] 60 (28, 32)
['Camille' 'Zyra' 'opp'] 60 (21, 39)
['Rengar' 'Varus' 'opp'] 60 (29, 31)
['Rengar' 'Shen' 'same'] 60 (30,)
['Leblanc' 'Ryze' 'opp'] 59 (25, 34)
['Camille' 'Shen' 'sa

['Jhin' 'Rengar' 'opp'] 48 (24, 24)
['Elise' 'Zyra' 'same'] 48 (32,)
['Rumble' 'Shen' 'opp'] 48 (19, 29)
['LeeSin' 'Rengar' 'opp'] 48 (27, 21)
['Elise' 'Zyra' 'opp'] 48 (19, 29)
['Graves' 'Syndra' 'opp'] 47 (24, 23)
['Elise' 'Leblanc' 'opp'] 47 (24, 23)
['Jayce' 'Ryze' 'same'] 47 (24,)
['Jayce' 'Maokai' 'same'] 47 (25,)
['Karma' 'Ryze' 'opp'] 47 (28, 19)
['Khazix' 'Zyra' 'same'] 47 (26,)
['Camille' 'Orianna' 'opp'] 47 (20, 27)
['Maokai' 'Ryze' 'same'] 47 (27,)
['Karma' 'Maokai' 'opp'] 46 (19, 27)
['Camille' 'Jayce' 'same'] 46 (26,)
['Cassiopeia' 'Rengar' 'same'] 46 (27,)
['Camille' 'Ezreal' 'opp'] 46 (29, 17)
['Elise' 'Varus' 'opp'] 46 (25, 21)
['Leblanc' 'Ryze' 'same'] 46 (29,)
['Leblanc' 'Shen' 'opp'] 46 (18, 28)
['Camille' 'Cassiopeia' 'opp'] 46 (22, 24)
['Karma' 'Varus' 'opp'] 46 (20, 26)
['Ashe' 'Karma' 'same'] 46 (24,)
['Jayce' 'Zyra' 'opp'] 45 (19, 26)
['Khazix' 'Ryze' 'same'] 45 (29,)
['Cassiopeia' 'Malzahar' 'opp'] 45 (16, 29)
['Ashe' 'Ryze' 'opp'] 45 (22, 23)
['Karma' 'Ryze' 

['Annie' 'Lissandra' 'same'] 20 (13,)
['JarvanIV' 'Maokai' 'same'] 20 (14,)
['Kalista' 'Lissandra' 'same'] 20 (12,)
['Morgana' 'Thresh' 'opp'] 20 (10, 10)
['Morgana' 'Rumble' 'opp'] 20 (14, 6)
['Irelia' 'Lissandra' 'same'] 20 (13,)
['JarvanIV' 'Morgana' 'opp'] 20 (5, 15)
['Morgana' 'Vi' 'same'] 20 (11,)
['Annie' 'Sivir' 'same'] 20 (13,)
['Kassadin' 'Lissandra' 'opp'] 20 (10, 10)
['Janna' 'RekSai' 'opp'] 20 (10, 10)
['Kassadin' 'Sivir' 'same'] 20 (10,)
['Annie' 'Rumble' 'opp'] 20 (12, 8)
['Annie' 'JarvanIV' 'same'] 20 (12,)
['Corki' 'JarvanIV' 'opp'] 20 (9, 11)
['Janna' 'Thresh' 'opp'] 19 (6, 13)
['Maokai' 'Zed' 'same'] 19 (11,)
['JarvanIV' 'Vi' 'opp'] 19 (6, 13)
['Leblanc' 'Lissandra' 'opp'] 19 (12, 7)
['Rumble' 'Thresh' 'opp'] 19 (11, 8)
['Irelia' 'RekSai' 'opp'] 19 (10, 9)
['Graves' 'RekSai' 'same'] 19 (10,)
['Morgana' 'Nidalee' 'opp'] 19 (10, 9)
['Annie' 'Maokai' 'same'] 19 (11,)
['JarvanIV' 'Zed' 'opp'] 19 (6, 13)
['JarvanIV' 'Maokai' 'opp'] 19 (6, 13)
-----------------------------

['Chogath' 'Xayah' 'same'] 24 (13,)
['Janna' 'Taliyah' 'same'] 23 (12,)
['Lulu' 'Shen' 'opp'] 23 (12, 11)
['Janna' 'Syndra' 'same'] 23 (16,)
['Galio' 'Gragas' 'opp'] 23 (7, 16)
['Lulu' 'Rakan' 'same'] 23 (16,)
['Chogath' 'Ryze' 'opp'] 23 (15, 8)
['Gragas' 'Xayah' 'opp'] 23 (15, 8)
['Chogath' 'Syndra' 'opp'] 23 (11, 12)
['Gragas' 'Rakan' 'same'] 23 (11,)
['Jayce' 'Lulu' 'opp'] 23 (12, 11)
['Gragas' 'Kalista' 'same'] 22 (7,)
['Galio' 'Gnar' 'same'] 22 (12,)
['Rakan' 'Shen' 'opp'] 22 (10, 12)
['KogMaw' 'Sejuani' 'opp'] 22 (10, 12)
['Chogath' 'Shen' 'opp'] 22 (10, 12)
['Shen' 'Xayah' 'same'] 22 (14,)
['Kalista' 'Syndra' 'opp'] 22 (9, 13)
['Janna' 'Shen' 'opp'] 22 (12, 10)
['Gragas' 'KogMaw' 'opp'] 22 (17, 5)
['Jayce' 'Lulu' 'same'] 22 (10,)
['Lulu' 'Ryze' 'same'] 22 (15,)
['JarvanIV' 'Ryze' 'opp'] 22 (14, 8)
['Ryze' 'Sejuani' 'same'] 21 (12,)
['Kalista' 'Taliyah' 'opp'] 21 (11, 10)
['Ryze' 'Sejuani' 'opp'] 21 (7, 14)
['Jayce' 'Shen' 'opp'] 21 (10, 11)
['Galio' 'Taliyah' 'opp'] 21 (10, 11)


In [13]:
game_counts, champ_counts, champ_pair_counts = get_counts(data_condensed, major_regions, 0.20, 'picks')

In [14]:
print_sorted(champ_counts, game_counts)

EULCS 2015 Spring (91 games)
Graves 48 [24]
Ahri 43 [22]
JarvanIV 42 [20]
Annie 40 [19]
RekSai 39 [16]
Gnar 36 [22]
Corki 36 [13]
Thresh 33 [15]
Maokai 33 [18]
Lulu 32 [15]
Leblanc 30 [13]
Sivir 28 [16]
Morgana 28 [16]
Leona 26 [13]
LeeSin 24 [9]
Lissandra 24 [11]
Janna 24 [11]
Nidalee 19 [14]
-----------------------------------------------
EULCS 2015 Summer (91 games)
Gragas 60 [34]
Alistar 53 [25]
RekSai 51 [25]
Sivir 40 [22]
Corki 38 [19]
Kalista 31 [19]
Rumble 31 [18]
Thresh 30 [17]
Shen 25 [10]
Azir 25 [11]
Maokai 25 [11]
Morgana 23 [13]
Nautilus 23 [10]
Lulu 22 [13]
Viktor 22 [8]
Gnar 21 [12]
Ekko 21 [8]
Varus 20 [12]
-----------------------------------------------
EULCS 2016 Spring (90 games)
Lucian 56 [29]
Corki 50 [27]
Braum 42 [18]
Elise 41 [17]
Alistar 40 [20]
Poppy 34 [17]
LeeSin 33 [16]
Thresh 32 [18]
Graves 30 [16]
Ezreal 28 [9]
Kalista 28 [18]
Nidalee 27 [18]
Ahri 25 [11]
Lissandra 25 [13]
Fiora 24 [10]
Gragas 22 [15]
Viktor 21 [11]
Trundle 21 [10]
RekSai 20 [6]
Nautilus

In [15]:
print_sorted(champ_pair_counts, game_counts)

EULCS 2015 Spring (91 games)
['RekSai' 'Zed' 'same'] 32 (17,)
['Ahri' 'RekSai' 'opp'] 31 (19, 12)
['Gnar' 'RekSai' 'same'] 30 (21,)
['JarvanIV' 'RekSai' 'opp'] 30 (16, 14)
['Gnar' 'Lissandra' 'opp'] 28 (14, 14)
['Kassadin' 'Lissandra' 'opp'] 26 (15, 11)
['Ahri' 'Zed' 'opp'] 26 (13, 13)
['Lissandra' 'RekSai' 'opp'] 26 (15, 11)
['LeeSin' 'RekSai' 'opp'] 25 (14, 11)
['Graves' 'RekSai' 'opp'] 25 (15, 10)
['Gnar' 'Kassadin' 'opp'] 25 (12, 13)
['RekSai' 'Zed' 'opp'] 24 (8, 16)
['Lissandra' 'Zed' 'opp'] 24 (10, 14)
['Kassadin' 'Zed' 'opp'] 24 (10, 14)
['Graves' 'Zed' 'opp'] 24 (11, 13)
['JarvanIV' 'Lissandra' 'same'] 24 (15,)
['Ahri' 'Graves' 'same'] 24 (15,)
['Kassadin' 'RekSai' 'opp'] 23 (16, 7)
['Gnar' 'Zed' 'same'] 23 (16,)
['Gnar' 'JarvanIV' 'opp'] 23 (12, 11)
['JarvanIV' 'Kassadin' 'opp'] 23 (11, 12)
['Leblanc' 'RekSai' 'opp'] 23 (12, 11)
['Nidalee' 'Zed' 'opp'] 23 (12, 11)
['Annie' 'RekSai' 'opp'] 23 (13, 10)
['Graves' 'JarvanIV' 'opp'] 22 (12, 10)
['Leblanc' 'RekSai' 'same'] 22 (12,)


['Rumble' 'Thresh' 'opp'] 27 (15, 12)
['Gnar' 'Lulu' 'same'] 27 (11,)
['JarvanIV' 'Thresh' 'same'] 27 (12,)
['RekSai' 'Xerath' 'same'] 27 (16,)
['Lissandra' 'Thresh' 'opp'] 26 (11, 15)
['Maokai' 'RekSai' 'same'] 26 (15,)
['Gnar' 'RekSai' 'same'] 26 (15,)
['Maokai' 'Rumble' 'opp'] 26 (14, 12)
['JarvanIV' 'Sivir' 'same'] 26 (16,)
['Corki' 'LeeSin' 'opp'] 26 (14, 12)
['Rumble' 'Xerath' 'same'] 26 (13,)
['Lissandra' 'Sivir' 'same'] 26 (19,)
-----------------------------------------------
LCK 2015 Summer (215 games)
['Gragas' 'RekSai' 'opp'] 112 (61, 51)
['Gragas' 'Kalista' 'same'] 111 (59,)
['Alistar' 'Kalista' 'opp'] 108 (62, 46)
['Kalista' 'RekSai' 'opp'] 104 (50, 54)
['Alistar' 'Gragas' 'opp'] 103 (49, 54)
['Kalista' 'Ryze' 'same'] 89 (53,)
['Kalista' 'Maokai' 'opp'] 86 (32, 54)
['Gragas' 'Kalista' 'opp'] 84 (50, 34)
['Azir' 'Kalista' 'opp'] 83 (45, 38)
['Gragas' 'Maokai' 'opp'] 83 (35, 48)
['Alistar' 'RekSai' 'opp'] 79 (40, 39)
['Maokai' 'RekSai' 'opp'] 79 (45, 34)
['Alistar' 'RekSai' 

['Malzahar' 'Rengar' 'same'] 82 (46,)
['Ashe' 'Zyra' 'opp'] 81 (36, 45)
['Leblanc' 'Varus' 'opp'] 79 (35, 44)
['Malzahar' 'Varus' 'same'] 79 (49,)
['Camille' 'Ryze' 'opp'] 78 (43, 35)
['Jhin' 'Rengar' 'same'] 77 (47,)
['Jayce' 'Rengar' 'opp'] 77 (39, 38)
['Rengar' 'Ryze' 'opp'] 76 (38, 38)
['Ashe' 'Malzahar' 'same'] 76 (45,)
['Karma' 'Varus' 'same'] 75 (49,)
['Camille' 'Nautilus' 'opp'] 75 (28, 47)
['Rengar' 'Zyra' 'same'] 75 (49,)
['Karma' 'Zyra' 'opp'] 73 (29, 44)
['Camille' 'Maokai' 'same'] 73 (44,)
['Varus' 'Zyra' 'same'] 73 (41,)
['Malzahar' 'Ryze' 'same'] 73 (39,)
['Camille' 'Shen' 'opp'] 71 (29, 42)
['Jayce' 'Malzahar' 'same'] 71 (44,)
['Camille' 'Jhin' 'opp'] 71 (29, 42)
['Camille' 'Syndra' 'opp'] 71 (34, 37)
['Rumble' 'Varus' 'opp'] 71 (38, 33)
['Maokai' 'Rengar' 'opp'] 70 (38, 32)
['Leblanc' 'Rengar' 'opp'] 70 (32, 38)
['Graves' 'Malzahar' 'same'] 70 (44,)
['Maokai' 'Varus' 'opp'] 69 (35, 34)
['Camille' 'Zyra' 'same'] 69 (36,)
['Ryze' 'Varus' 'opp'] 69 (28, 41)
['Camille' 'Ka

['Malzahar' 'Nautilus' 'opp'] 52 (27, 25)
['Graves' 'Varus' 'opp'] 51 (32, 19)
['Leblanc' 'Ryze' 'same'] 51 (33,)
['Ashe' 'Camille' 'same'] 51 (28,)
['Malzahar' 'Zyra' 'opp'] 51 (30, 21)
['Jayce' 'Shen' 'opp'] 51 (30, 21)
['Karma' 'Ryze' 'same'] 51 (26,)
['Graves' 'Khazix' 'opp'] 50 (29, 21)
['Ashe' 'Nautilus' 'opp'] 50 (22, 28)
['Jhin' 'Khazix' 'same'] 50 (28,)
['Leblanc' 'Nautilus' 'opp'] 50 (19, 31)
['Jayce' 'Malzahar' 'same'] 50 (28,)
['Jayce' 'Maokai' 'opp'] 50 (20, 30)
['Graves' 'Shen' 'same'] 50 (30,)
['Rengar' 'Rumble' 'opp'] 50 (21, 29)
['Rengar' 'Zyra' 'same'] 50 (26,)
['Jayce' 'Jhin' 'opp'] 50 (25, 25)
['Malzahar' 'Maokai' 'same'] 50 (31,)
['Karma' 'Leblanc' 'same'] 50 (31,)
['Malzahar' 'Nautilus' 'same'] 49 (30,)
['Ryze' 'Zyra' 'opp'] 49 (29, 20)
['Malzahar' 'Rumble' 'opp'] 49 (25, 24)
['Camille' 'Rumble' 'same'] 49 (25,)
['Karma' 'Malzahar' 'same'] 49 (30,)
['Jayce' 'Khazix' 'opp'] 49 (26, 23)
['Camille' 'Lulu' 'same'] 49 (33,)
['Jayce' 'Varus' 'same'] 49 (31,)
['Ashe' 'Ry

['Elise' 'Ezreal' 'same'] 20 (11,)
['Nidalee' 'RekSai' 'opp'] 20 (8, 12)
['Jayce' 'Jhin' 'same'] 20 (9,)
['Olaf' 'Ryze' 'opp'] 20 (11, 9)
['Caitlyn' 'Elise' 'opp'] 20 (11, 9)
['Jayce' 'Nidalee' 'same'] 20 (10,)
['Nidalee' 'Viktor' 'opp'] 20 (10, 10)
['Caitlyn' 'Karma' 'opp'] 20 (12, 8)
['Elise' 'LeeSin' 'opp'] 20 (7, 13)
['Caitlyn' 'Ryze' 'opp'] 20 (10, 10)
['Caitlyn' 'Jayce' 'same'] 19 (9,)
['Jhin' 'Karma' 'opp'] 19 (13, 6)
['Karma' 'LeeSin' 'opp'] 19 (7, 12)
['Elise' 'Karma' 'same'] 19 (9,)
['Karma' 'Zyra' 'opp'] 19 (8, 11)
['Jhin' 'Karma' 'same'] 19 (10,)
['Jayce' 'Syndra' 'same'] 19 (12,)
['AurelionSol' 'Ryze' 'opp'] 19 (13, 6)
['Ezreal' 'Syndra' 'same'] 19 (11,)
['Jayce' 'Olaf' 'opp'] 19 (13, 6)
['Nidalee' 'Sivir' 'same'] 19 (10,)
['Jayce' 'Karma' 'opp'] 19 (12, 7)
['Kennen' 'Nidalee' 'opp'] 19 (11, 8)
['Kennen' 'Syndra' 'same'] 19 (8,)
['Poppy' 'Syndra' 'opp'] 19 (9, 10)
['Ezreal' 'Syndra' 'opp'] 19 (5, 14)
['Nidalee' 'TahmKench' 'opp'] 19 (9, 10)
['Caitlyn' 'Olaf' 'same'] 19 (12

In [16]:
# care about this one most
game_counts, champ_counts, champ_pair_counts = get_counts(data_condensed, major_regions, 0.20)

In [17]:
print_sorted(champ_counts, game_counts)

EULCS 2015 Spring (91 games)
['Zed' 'b'] 49 [23]
['Graves' 'p'] 48 [24]
['Ahri' 'p'] 43 [22]
['JarvanIV' 'p'] 42 [20]
['Annie' 'p'] 40 [19]
['RekSai' 'p'] 39 [16]
['Kassadin' 'b'] 39 [21]
['Gnar' 'p'] 36 [22]
['Corki' 'p'] 36 [13]
['Cassiopeia' 'b'] 34 [22]
['RekSai' 'b'] 33 [14]
['Thresh' 'p'] 33 [15]
['Maokai' 'p'] 33 [18]
['Lulu' 'p'] 32 [15]
['Nidalee' 'b'] 30 [15]
['Leblanc' 'p'] 30 [13]
['Sivir' 'p'] 28 [16]
['Leblanc' 'b'] 28 [14]
['Morgana' 'p'] 28 [16]
['Lissandra' 'b'] 27 [14]
['Leona' 'p'] 26 [13]
['LeeSin' 'p'] 24 [9]
['Lissandra' 'p'] 24 [11]
['Janna' 'p'] 24 [11]
['Azir' 'b'] 20 [6]
['Gnar' 'b'] 20 [7]
['Nidalee' 'p'] 19 [14]
['Ahri' 'b'] 19 [12]
-----------------------------------------------
EULCS 2015 Summer (91 games)
['Gragas' 'p'] 60 [34]
['Alistar' 'p'] 53 [25]
['Ryze' 'b'] 52 [22]
['RekSai' 'p'] 51 [25]
['Kalista' 'b'] 47 [19]
['Sivir' 'p'] 40 [22]
['Corki' 'p'] 38 [19]
['Kalista' 'p'] 31 [19]
['Sivir' 'b'] 31 [14]
['Rumble' 'p'] 31 [18]
['Thresh' 'p'] 30 [17]
['R

In [18]:
print_sorted(champ_pair_counts, game_counts)

EULCS 2015 Spring (91 games)
[('Corki', 'p') ('Graves', 'p') 'opp'] 19 [ 5 14]
-----------------------------------------------
EULCS 2015 Summer (91 games)
[('Gragas', 'p') ('RekSai', 'p') 'opp'] 35 (18, 17)
[('Gragas', 'p') ('Ryze', 'b') 'same'] 21 (10,)
[('Alistar', 'p') ('RekSai', 'p') 'opp'] 20 (9, 11)
-----------------------------------------------
EULCS 2016 Spring (90 games)
[('Corki', 'p') ('Lulu', 'b') 'opp'] 24 (14, 10)
[('Elise', 'p') ('Lulu', 'b') 'opp'] 24 (13, 11)
[('Gangplank', 'b') ('Lulu', 'b') 'same'] 24 (12,)
[('Gangplank', 'b') ('Lucian', 'p') 'same'] 23 (14,)
[('Kalista', 'b') ('Lucian', 'p') 'opp'] 22 (10, 12)
[('Corki', 'p') ('Kalista', 'b') 'same'] 21 (12,)
[('Lucian', 'p') ('Lulu', 'b') 'same'] 21 (8,)
[('Kalista', 'b') ('Lulu', 'b') 'opp'] 20 (10, 10)
[('Kalista', 'b') ('Lucian', 'p') 'same'] 20 (9,)
[('Elise', 'p') ('Gangplank', 'b') 'same'] 20 (14,)
[('Corki', 'p') ('Gangplank', 'b') 'opp'] 19 (10, 9)
[('Corki', 'p') ('Lucian', 'p') 'opp'] 19 (10, 9)
[('Alis

[('Kalista', 'b') ('Tristana', 'p') 'same'] 26 (15,)
[('Kalista', 'b') ('Xayah', 'b') 'same'] 23 (12,)
[('Jayce', 'b') ('Kalista', 'b') 'opp'] 22 (11, 11)
[('Chogath', 'p') ('Kalista', 'b') 'opp'] 22 (11, 11)
[('Kalista', 'b') ('Sejuani', 'b') 'same'] 22 (14,)
[('Kalista', 'b') ('Lulu', 'p') 'opp'] 21 (11, 10)
[('Galio', 'b') ('Xayah', 'b') 'opp'] 21 (11, 10)
[('Gragas', 'p') ('Kalista', 'b') 'opp'] 21 (15, 6)
[('Kalista', 'b') ('Xayah', 'b') 'opp'] 21 (7, 14)
[('JarvanIV', 'b') ('Kalista', 'b') 'opp'] 21 (12, 9)
[('Kalista', 'b') ('Lulu', 'b') 'opp'] 20 (8, 12)
[('Kalista', 'b') ('Xayah', 'p') 'opp'] 20 (8, 12)
[('Galio', 'b') ('Sejuani', 'p') 'opp'] 20 (9, 11)
[('Galio', 'b') ('Kalista', 'b') 'same'] 19 (12,)
[('Galio', 'b') ('Lulu', 'p') 'same'] 19 (9,)
[('JarvanIV', 'p') ('Sejuani', 'p') 'opp'] 19 (12, 7)
[('Kalista', 'b') ('Tristana', 'p') 'opp'] 19 (9, 10)
[('Chogath', 'p') ('Kalista', 'b') 'same'] 19 (9,)
[('Janna', 'p') ('Tristana', 'p') 'opp'] 18 (13, 5)
[('Kalista', 'b') ('Ra

# K-means
Below we attempt to cluster various "metas," which we distinguish based on region/season pair, based on how many champions they have in common. This was done by finding the percentage of games each champion appeared during a season in a given region, then using this percentage for each champion as coordinates for that meta. We then applied k-means to the data under various means of defining these coordinates (i.e. whether we counted bans as being the same as picks, as being separate, or not at all) and attempted to create elbow plots for the optimal k.

In [19]:
# set value of s here to keep consistent between runs
# we want to only keep champions that were at least somewhat frequent, otherwise champions 
# that were picked only once or twice would add another dimension without contributing meaningful information
s = 0.10

In [20]:
# get counts, convert to data vectors to use for clustering
game_counts, champ_counts, champ_pair_counts = get_counts(data_less, all_regions, s, 'presence')
vecs, champ_list, regions = get_vecs(champ_counts, game_counts)
print(vecs.shape)

k_vals = list(range(1, 18))
errs = [] # mean squared distance from centroid for each k (not technically an "error" but these are my variables and I name these what I want)
n = 100 # number of times to run for each value of k

for k in k_vals:
    err = []
    for j in range(n):
        centroids, clusters = None, None
        iters = 0
        while clusters is None:
            iters += 1
            centroids, clusters = k_means(vecs, k)
            if iters >= 10: # skip if cannot quickly find initial conditions that result in an empty cluster, bc I would like this to complete before I am dead
                print('Skipping during ' + str(k) + '-' + str(j))
                continue
        for i in range(len(clusters)): # add square distance from corresponding centroid for each data point
            err.append(np.linalg.norm(vecs[i, :] - centroids[clusters[i]])**2)
    errs.append(np.mean(err))
    
plt.plot(k_vals, errs)

(71, 125)


NameError: name 'math' is not defined

In [ ]:
# pretend this cell has the same comments as previous one

game_counts, champ_counts, champ_pair_counts = get_counts(data_less, all_regions, 0.25, 'picks')
vecs, champ_list, regions = get_vecs(champ_counts, game_counts)
print(vecs.shape)

k_vals = list(range(1, 18))
errs = []
n = 100

for k in k_vals:
    err = []
    for j in range(n):
        centroids, clusters = None, None
        iters = 0
        while clusters is None:
            iters += 1
            centroids, clusters = k_means(vecs, k)
            if iters >= 10:
                print('Skipping during ' + str(k) + '-' + str(j))
                continue
        for i in range(len(clusters)):
            err.append(np.linalg.norm(vecs[i, :] - centroids[clusters[i]])**2)
    errs.append(np.mean(err))
    
plt.plot(k_vals, errs)

In [ ]:
# see comment at top of previous cell

game_counts, champ_counts, champ_pair_counts = get_counts(data_less, all_regions, s)
vecs, champ_list, regions = get_vecs(champ_counts, game_counts)
print(vecs.shape)

k_vals = list(range(1, 18))
errs = []
n = 100

for k in k_vals:
    err = []
    for j in range(n):
        centroids, clusters = None, None
        iters = 0
        while clusters is None:
            iters += 1
            centroids, clusters = k_means(vecs, k)
            if iters >= 10:
                print('Skipping during ' + str(k) + '-' + str(j))
                continue
        for i in range(len(clusters)):
            err.append(np.linalg.norm(vecs[i, :] - centroids[clusters[i]])**2)
    errs.append(np.mean(err))
    
plt.plot(k_vals, errs)

In the following cells, we take k=8, since the rate at which the mean square distance decreased seemed to slow somewhere around this point.

In [ ]:
# get counts, convert to data vectors to use for clustering
game_counts, champ_counts, champ_pair_counts = get_counts(data_less, all_regions, s, 'presence')
vecs, champ_list, regions = get_vecs(champ_counts, game_counts)

n = 20 # number of candidate clusters to have
cluster_candidates = [] 
errs = [] # "error," as defined in previous cells
for j in range(n):
    err = []
    centroids, clusters = None, None
    iters = 0
    while clusters is None: # keep trying until some initialization doesn't cause a cluster to become empty
        centroids, clusters = k_means(vecs, 8)
    cluster_candidates.append(clusters)
    # calculate "errors" (distances) squared for each point to their corresponding centroid
    for i in range(len(clusters)): 
        err.append(np.linalg.norm(vecs[i, :] - centroids[clusters[i]])**2)
    errs.append(np.mean(err))

# pick clustering that minimizes the mean squared distance
idx = np.argmin(errs)
clusters = cluster_candidates[idx]

for i in range(8):
    print('Cluster {}:'.format(i))
    for j in range(len(clusters)):
        if clusters[j] == i:
            print(regions[j])
    print('-------------')
    
grid_plot_clusters(regions, clusters, '')

In [ ]:
# see top of three cells ago

game_counts, champ_counts, champ_pair_counts = get_counts(data_less, all_regions, s, 'picks')
vecs, champ_list, regions = get_vecs(champ_counts, game_counts)

n = 20 
cluster_candidates = []
errs = []
for j in range(n):
    err = []
    centroids, clusters = None, None
    iters = 0
    while clusters is None:
        centroids, clusters = k_means(vecs, 8)
    cluster_candidates.append(clusters)
    for i in range(len(clusters)):
        err.append(np.linalg.norm(vecs[i, :] - centroids[clusters[i]])**2)
    errs.append(np.mean(err))
idx = np.argmin(errs)
clusters = cluster_candidates[idx]

for i in range(8):
    print('Cluster {}:'.format(i))
    for j in range(len(clusters)):
        if clusters[j] == i:
            print(regions[j])
    print('-------------')
    
grid_plot_clusters(regions, clusters, '')

In [ ]:
# same comments as previous cells

game_counts, champ_counts, champ_pair_counts = get_counts(data_less, all_regions, s)
vecs, champ_list, regions = get_vecs(champ_counts, game_counts)

n = 20
cluster_candidates = []
errs = []
for j in range(n):
    err = []
    centroids, clusters = None, None
    iters = 0
    while clusters is None:
        centroids, clusters = k_means(vecs, 8)
    cluster_candidates.append(clusters)
    for i in range(len(clusters)):
        err.append(np.linalg.norm(vecs[i, :] - centroids[clusters[i]])**2)
    errs.append(np.mean(err))
idx = np.argmin(errs)
clusters = cluster_candidates[idx]

for i in range(8):
    print('Cluster {}:'.format(i))
    for j in range(len(clusters)):
        if clusters[j] == i:
            print(regions[j])
    print('-------------')
    
grid_plot_clusters(regions, clusters, '')